In [ ]:
print("SCRIPT IS RUNNING")

import os
import mne
import numpy as np
import pandas as pd
from mne.time_frequency import psd_array_welch

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(4, 150)]

def spectral_entropy(psd, freqs, fmin=1, fmax=45, normalize=True):
    idx = np.logical_and(freqs >= fmin, freqs <= fmax)
    psd_band = psd[:, :, idx]
    psd_norm = psd_band / psd_band.sum(axis=-1, keepdims=True)
    se = -np.sum(psd_norm * np.log2(psd_norm + 1e-12), axis=-1)
    if normalize:
        se /= np.log2(psd_norm.shape[-1])
    return se

for SUBJ in SUBJECTS:
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    subj_num = SUBJ.split('-')[1]  # extract just the number, e.g., "001"
    epo_file = os.path.join(SUBJ_DIR, f"epo_{subj_num}_raw.fif")  # correct filename

    if not os.path.exists(epo_file):
        print("Checking:", epo_file)
        continue
    else:
        print("FOUND:", epo_file)

    # Load epochs
    epochs = mne.read_epochs(epo_file, preload=True, verbose=False)
    sfreq = epochs.info["sfreq"]
    ch_names = epochs.ch_names

    # Compute PSD with 4s epochs and 50% overlap
    n_per_seg = int(sfreq * 4)
    noverlap = n_per_seg // 2

    psds, freqs = psd_array_welch(
        epochs.get_data(), sfreq=sfreq, fmin=1, fmax=45,
        n_fft=1024,
        n_per_seg=n_per_seg,
        n_overlap=noverlap,
        window='hann',
        verbose=False
    )

    # Compute spectral entropy per epoch/channel
    H = spectral_entropy(psds, freqs)

    # Per-epoch/channel CSV (long format)
    df_entropy = pd.DataFrame(H, columns=ch_names)
    df_entropy.insert(0, 'epoch', np.arange(len(df_entropy)))
    df_long = df_entropy.melt(id_vars=['epoch'], var_name='channel', value_name='spec_entropy')
    output_path = os.path.join(SUBJ_DIR, f"{SUBJ}_spectral_entropy.csv")
    df_long.to_csv(output_path, index=False)

    # Mean per subject across all epochs/channels
    subject_mean = H.mean()
    mean_output_path = os.path.join(SUBJ_DIR, f"{SUBJ}_spectral_entropy_mean.csv")
    pd.DataFrame({'subject': [SUBJ], 'mean_spec_entropy': [subject_mean]}).to_csv(mean_output_path, index=False)

    print(f"{SUBJ}: Per-epoch/channel saved, mean spectral entropy = {subject_mean:.4f}")


In [ ]:
import os
import pandas as pd

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(1, 51)]  # first 50 subjects

summary_list = []

for sub in SUBJECTS:
    mean_file = os.path.join(BASE_DIR, sub, f"{sub}_spectral_entropy_mean.csv")
    if os.path.exists(mean_file):
        df = pd.read_csv(mean_file)
        summary_list.append(df)
    else:
        print(f"Missing file: {mean_file}")

# Combine all into one DataFrame
summary_df = pd.concat(summary_list, ignore_index=True)

# Save combined CSV
output_path = os.path.join(BASE_DIR, "mean_spectral_entropy_first50.csv")
summary_df.to_csv(output_path, index=False)
print(f"Saved summary CSV to {output_path}")


### Spectral Entropy Frontal-Temporal-Central Decreases in Cognitive Impairment

In [7]:
import os
import pandas as pd
import numpy as np

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(4, 150)]

# Define regions
REGIONS = {
    'frontal': ['Fp1', 'Fz', 'F3', 'F7', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'F6', 'AF8', 'AF4', 'F2', 'FCz'],
    'temporal': ['T7','TP7','FT7','FT8', 'TP9', 'T8', 'TP10', 'TP8'],
    'central': ['C3','C4','CP1','CP2', 'C1', 'C5', 'Cz', 'C6', 'C2']
}

all_region_means = []

for SUBJ in SUBJECTS:
    csv_file = os.path.join(BASE_DIR, SUBJ, f"{SUBJ}_spectral_entropy.csv")
    if not os.path.exists(csv_file):
        print(f"{SUBJ}: missing CSV")
        continue

    df = pd.read_csv(csv_file)
    df['channel'] = df['channel'].str.upper()

    region_means = {'subject': SUBJ}
    for region, ch_list in REGIONS.items():
        idxs = df['channel'].isin(ch_list)
        if idxs.any():
            region_means[region] = df.loc[idxs, 'spec_entropy'].mean()
        else:
            region_means[region] = np.nan

    all_region_means.append(region_means)

# Combine into a single DataFrame
df_regions = pd.DataFrame(all_region_means)

# Save summary
summary_file = os.path.join(BASE_DIR, "spectral_entropy_regions_mean.csv")
df_regions.to_csv(summary_file, index=False)
print(f"Saved regional spectral entropy summary → {summary_file}")


Saved regional spectral entropy summary → C:\Users\User\Documents\EEG_Project\rEEG\spectral_entropy_regions_mean.csv


In [10]:
import pandas as pd
from scipy.stats import ttest_ind, spearmanr
import os

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"

# Load regional spectral entropy means
df_entropy = pd.read_csv(os.path.join(BASE_DIR, "spectral_entropy_regions_mean.csv"))

# Load participant metadata
meta = pd.read_csv(os.path.join(BASE_DIR, "participants.csv"))

# Merge on subject ID
df = df_entropy.merge(
    meta[['participant_id', 'GROUP', 'MOCA']],
    left_on='subject', right_on='participant_id',
    how='inner'
)

# Filter PD group and drop missing MoCA
df = df[df['GROUP'] == 'PD'].dropna(subset=['MOCA']).copy()

# Define impaired vs unimpaired
df['impaired'] = df['MOCA'] < 26

# Regions to analyze
REGIONS = ['frontal', 'temporal', 'central']

# Collect stats
results = []
for region in REGIONS:
    impaired_vals = df.loc[df['impaired'], region]
    unimpaired_vals = df.loc[~df['impaired'], region]

    # T-test
    t_stat, p_group = ttest_ind(impaired_vals, unimpaired_vals, equal_var=False)

    # Spearman correlation
    rho, p_corr = spearmanr(df[region], df['MOCA'])

    results.append({
        'Region': region.capitalize(),
        'N_total': len(df),
        'N_impaired': impaired_vals.size,
        'N_unimpaired': unimpaired_vals.size,
        'T_stat': t_stat,
        'P_group': p_group,
        'Rho_MoCA': rho,
        'P_corr': p_corr,
        'Impaired_mean': impaired_vals.mean(),
        'Impaired_std': impaired_vals.std(),
        'Unimpaired_mean': unimpaired_vals.mean(),
        'Unimpaired_std': unimpaired_vals.std()
    })

# Convert to DataFrame and sort by correlation p-value
df_results = pd.DataFrame(results).sort_values('P_corr')

# Save CSV
df_results.to_csv(os.path.join(BASE_DIR, "spectral_entropy_regions_stats_PD.csv"), index=False)

# -----------------------------
# Nicely formatted console output (full precision)
# -----------------------------
print("\n=== Spectral Entropy Stats by Region (PD Only) ===\n")
for _, row in df_results.iterrows():
    print(f"Region: {row['Region']}")
    print(f"  Total N: {row['N_total']} | Impaired: {row['N_impaired']} | Unimpaired: {row['N_unimpaired']}")
    print(f"  Impaired Mean ± SD: {row['Impaired_mean']} ± {row['Impaired_std']}")
    print(f"  Unimpaired Mean ± SD: {row['Unimpaired_mean']} ± {row['Unimpaired_std']}")
    print(f"  T-test: t = {row['T_stat']}, p = {row['P_group']}")
    print(f"  Spearman correlation with MoCA: rho = {row['Rho_MoCA']}, p = {row['P_corr']}\n")



=== Spectral Entropy Stats by Region (PD Only) ===

Region: Temporal
  Total N: 97 | Impaired: 51 | Unimpaired: 46
  Impaired Mean ± SD: 0.7771782156888014 ± 0.06296397607508844
  Unimpaired Mean ± SD: 0.7993006501427018 ± 0.056866231241105805
  T-test: t = -1.8182449024049727, p = 0.07217807690636543
  Spearman correlation with MoCA: rho = 0.19465678200272193, p = 0.056054381817122016

Region: Central
  Total N: 97 | Impaired: 51 | Unimpaired: 46
  Impaired Mean ± SD: 0.7735591985304698 ± 0.06299938426322786
  Unimpaired Mean ± SD: 0.7947946530199556 ± 0.05443543400870729
  T-test: t = -1.7805372282217904, p = 0.07819036897892279
  Spearman correlation with MoCA: rho = 0.1747966765585994, p = 0.08681554741560375

Region: Frontal
  Total N: 97 | Impaired: 51 | Unimpaired: 46
  Impaired Mean ± SD: 0.7717178785251777 ± 0.06644660587427434
  Unimpaired Mean ± SD: 0.7934133295606058 ± 0.06006721089937489
  T-test: t = -1.6889481545383467, p = 0.09450919986994302
  Spearman correlation wit